In [ ]:
# Install required packages
!pip install spacy==3.7.2 sentence-transformers==2.2.2 scikit-learn==1.2.2 numpy==1.24.3
!python -m spacy download en_core_web_md

print("All packages installed successfully!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 58.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 73.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Create the client_scenarios.json file
import json

client_scenarios = {
  "scenarios": [
    {
      "id": 1,
      "business_domain": "Retail",
      "business_type": "Clothing Store",
      "client_input": "I want a way to sell my handmade clothes online. I also need to manage my stock and run sales sometimes.",
      "client_input_vagueness": "low",
      "required_follow_up": None,
      "recommended_platform": ["Web App", "Mobile App"],
      "recommended_features": ["E-commerce Cart", "Payment Gateway Integration (Stripe/PayPal)", "Inventory Management System", "Admin Dashboard", "Promo Code Engine"],
      "recommended_tech_stack": {
        "frontend": "React.js / Next.js",
        "backend": "Node.js / Python (Django)",
        "database": "PostgreSQL",
        "mobile": "React Native (for cost-effectiveness)"
      },
      "estimated_dev_time": "3-5 months",
      "estimated_cost_range": "$20,000 - $40,000"
    },
    {
      "id": 2,
      "business_domain": "Logistics",
      "business_type": "Local Courier Service",
      "client_input": "I need an app for my delivery guys. They should see where to go and mark delivered.",
      "client_input_vagueness": "medium",
      "required_follow_up": "Do you need real-time location tracking for your dispatchers to see all drivers on a map?",
      "recommended_platform": ["Mobile App (Driver)", "Web App (Dispatcher/Admin)"],
      "recommended_features": ["Real-time GPS Tracking", "Order Assignment & Management", "Proof of Delivery (Photo/Signature Capture)", "Admin Dashboard with Live Map View", "Push Notifications"],
      "recommended_tech_stack": {
        "mobile": "Flutter (Cross-platform for iOS & Android)",
        "backend": "Node.js with Socket.io for real-time features",
        "database": "MongoDB or Firebase Firestore for flexible data",
        "maps": "Google Maps Platform API"
      },
      "estimated_dev_time": "4-6 months",
      "estimated_cost_range": "$45,000 - $70,000"
    },
    {
      "id": 3,
      "business_domain": "Other",
      "business_type": "Startup",
      "client_input": "I have a great idea for social networking but I'm not sure how to build it.",
      "client_input_vagueness": "high",
      "required_follow_up": "Can you describe the core functionality? Is it about connecting people based on location, interests, or professional networks?",
      "recommended_platform": ["Mobile App", "Web App"],
      "recommended_features": ["User Profiles", "News Feed", "Real-time Messaging", "Media Upload", "Social Authentication (Login with Google/FB)"],
      "recommended_tech_stack": {
        "frontend": "React Native / React.js",
        "backend": "Node.js / Express.js",
        "database": "MongoDB",
        "realtime": "Socket.io or Pusher"
      },
      "estimated_dev_time": "6+ months",
      "estimated_cost_range": "$70,000+"
    },
    {
      "id": 4,
      "business_domain": "Food",
      "business_type": "Restaurant",
      "client_input": "I want to allow customers to order food online from my restaurant for both delivery and pickup.",
      "client_input_vagueness": "low",
      "required_follow_up": None,
      "recommended_platform": ["Web App", "Mobile App"],
      "recommended_features": ["Online Ordering System", "Menu Management", "Payment Processing", "Order Tracking", "Customer Reviews"],
      "recommended_tech_stack": {
        "frontend": "React.js / React Native",
        "backend": "Node.js / Express.js",
        "database": "MongoDB",
        "payment": "Stripe API"
      },
      "estimated_dev_time": "3-4 months",
      "estimated_cost_range": "$25,000 - $45,000"
    },
    {
      "id": 5,
      "business_domain": "Consultancy",
      "business_type": "Legal Services",
      "client_input": "I need a system to manage my client appointments, documents, and billing.",
      "client_input_vagueness": "medium",
      "required_follow_up": "Do you need client portal functionality for document sharing and communication?",
      "recommended_platform": ["Web App"],
      "recommended_features": ["Appointment Scheduling", "Client Portal", "Document Management", "Billing & Invoicing", "Time Tracking"],
      "recommended_tech_stack": {
        "frontend": "React.js",
        "backend": "Python/Django",
        "database": "PostgreSQL",
        "storage": "AWS S3 for document storage"
      },
      "estimated_dev_time": "4-5 months",
      "estimated_cost_range": "$30,000 - $50,000"
    }
  ]
}

# Save to a JSON file
with open('client_scenarios.json', 'w') as f:
    json.dump(client_scenarios, f, indent=2)

print("Dataset created successfully!")

Dataset created successfully!


In [ ]:
# NLP Processor Implementation
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the medium English model for better accuracy
nlp = spacy.load("en_core_web_md")

class NLPProcessor:
    def __init__(self):
        # Define keywords for each business domain for rule-based matching
        self.domain_keywords = {
            "retail": ["sell", "shop", "store", "product", "inventory", "cart", "payment", "ecommerce", "customer"],
            "logistics": ["deliver", "ship", "track", "driver", "fleet", "dispatch", "courier", "package"],
            "food": ["food", "restaurant", "menu", "order", "reservation", "delivery", "dish", "table"],
            "consultancy": ["schedule", "appointment", "client", "invoice", "bill", "meeting", "consult"],
            "social": ["social", "connect", "profile", "share", "post", "message", "network", "friend"]
        }

    def preprocess_text(self, text):
        """Basic text preprocessing."""
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        return text

    def analyze_input(self, user_input):
        """Analyzes user input to extract intent, entities, and vagueness score."""
        doc = nlp(user_input)
        processed_input = self.preprocess_text(user_input)

        # Rule-based Intent Classification
        intent_scores = {}
        for domain, keywords in self.domain_keywords.items():
            score = sum(1 for keyword in keywords if keyword in processed_input)
            intent_scores[domain] = score

        primary_intent = max(intent_scores, key=intent_scores.get) if max(intent_scores.values()) > 0 else "unknown"

        # Entity Extraction using SpaCy
        entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT', 'GPE', 'NORP']] # Organizations, Products, Locations, Nationalities
        # Add keywords as entities
        nouns_verbs = [token.lemma_ for token in doc if token.pos_ in ['NOUN', 'VERB']]
        entities.extend(nouns_verbs)

        # Heuristic for Vagueness Detection
        vagueness = "high"
        if len(doc.ents) > 1 or len(nouns_verbs) > 3: # If there are several entities or specific terms
            vagueness = "low"
        elif len(doc.ents) == 1 or len(nouns_verbs) == 2:
            vagueness = "medium"

        analysis_result = {
            "raw_input": user_input,
            "processed_input": processed_input,
            "primary_intent": primary_intent,
            "entities": list(set(entities)), # Remove duplicates
            "vagueness": vagueness
        }
        return analysis_result

    def generate_follow_up(self, analysis_result):
        """Generates a context-aware follow-up question based on the analysis."""
        intent = analysis_result['primary_intent']
        vagueness = analysis_result['vagueness']

        if vagueness == "high":
            if intent == "unknown":
                return "Can you tell me more about what kind of business you have and what you're trying to achieve?"
            else:
                return {
                    "retail": "What specific products are you looking to sell online? Do you need features like a shopping cart or inventory management?",
                    "logistics": "Are you looking for real-time tracking for your drivers and a dashboard for your dispatchers?",
                    "food": "Are you looking for an online ordering system, table reservations, or both?",
                    "consultancy": "Do you primarily need a scheduling system for your clients and an invoicing tool?",
                    "social": "Can you describe the core functionality? Is it about connecting people based on location, interests, or professional networks?"
                }.get(intent, "Can you provide more specific details about your project?")
        else:
            return None # No follow-up needed if input is clear

print("NLP Processor defined successfully!")

NLP Processor defined successfully!


In [ ]:
# Recommendation Engine Implementation
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

class RecommendationEngine:
    def __init__(self, data_path='client_scenarios.json'):
        self.nlp_processor = NLPProcessor()
        self.model = SentenceTransformer('all-MiniLM-L6-v2') # Lightweight but effective model
        self.scenarios = self.load_data(data_path)
        self.scenario_embeddings = self.precompute_embeddings()

    def load_data(self, data_path):
        with open(data_path, 'r') as f:
            data = json.load(f)
        return data['scenarios']

    def precompute_embeddings(self):
        """Precompute embeddings for all scenario client_inputs for fast similarity search."""
        texts = [f"{s['business_domain']} {s['business_type']} {s['client_input']}" for s in self.scenarios]
        return self.model.encode(texts)

    def find_most_similar_scenario(self, user_input_analysis):
        """Finds the most similar scenario using cosine similarity on embeddings."""
        # Create a query string from the analysis
        query_text = f"{user_input_analysis['primary_intent']} {' '.join(user_input_analysis['entities'])}"
        query_embedding = self.model.encode([query_text])

        # Calculate similarity scores
        similarities = cosine_similarity(query_embedding, self.scenario_embeddings)
        most_similar_index = np.argmax(similarities)
        highest_similarity_score = similarities[0][most_similar_index]

        # Only return a recommendation if similarity is above a threshold
        if highest_similarity_score > 0.3: # Tunable threshold
            return self.scenarios[most_similar_index], highest_similarity_score
        else:
            return None, highest_similarity_score

    def get_recommendation(self, user_input):
        """Main function to get a recommendation for a user input."""
        print(f"Analyzing your input: '{user_input}'")
        analysis = self.nlp_processor.analyze_input(user_input)
        print(f"Analysis: {analysis}")

        follow_up = self.nlp_processor.generate_follow_up(analysis)
        if follow_up:
            return {"status": "follow_up_required", "message": follow_up}

        # If no follow-up is needed, find a recommendation
        recommendation, score = self.find_most_similar_scenario(analysis)
        if recommendation:
            result = {
                "status": "recommendation",
                "analysis": analysis,
                "confidence_score": round(score, 2),
                "recommendation": {
                    "platform": recommendation['recommended_platform'],
                    "features": recommendation['recommended_features'],
                    "tech_stack": recommendation['recommended_tech_stack'],
                    "estimated_dev_time": recommendation['estimated_dev_time'],
                    "estimated_cost_range": recommendation['estimated_cost_range']
                }
            }
            return result
        else:
            return {"status": "unknown", "message": "I couldn't find a strong match for your request. Please try rephrasing with more details about your business and goals."}

print("Recommendation Engine defined successfully!")

Recommendation Engine defined successfully!


In [ ]:
# Main Application
from google.colab import output
output.enable_custom_widget_manager()

def main():
    print("\n" + "="*50)
    print("        AI Client Clearance Assistant")
    print("        Rayonix Solutions - Expert System")
    print("="*50)
    print("Hello! I'm here to help you identify the best software solution for your business.")
    print("Describe your business or idea in a few sentences.\n")

    engine = RecommendationEngine()

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("Thank you for using the AI Assistant. Goodbye!")
            break

        result = engine.get_recommendation(user_input)

        if result['status'] == 'follow_up_required':
            print(f"\nAI Assistant: {result['message']}\n")
        elif result['status'] == 'recommendation':
            print("\n" + "-"*30)
            print("RECOMMENDATION RESULTS:")
            print("-"*30)
            print(f"Confidence Score: {result['confidence_score']}")
            print(f"Recommended Platform(s): {', '.join(result['recommendation']['platform'])}")
            print("\nKey Suggested Features:")
            for feature in result['recommendation']['features']:
                print(f" - {feature}")
            print(f"\nEstimated Development Time: {result['recommendation']['estimated_dev_time']}")
            print(f"Estimated Cost Range: {result['recommendation']['estimated_cost_range']}")
            print("\n" + "-"*30 + "\n")
        elif result['status'] == 'unknown':
            print(f"\nAI Assistant: {result['message']}\n")

# Run the application
if __name__ == "__main__":
    main()


        AI Client Clearance Assistant
        Rayonix Solutions - Expert System
Hello! I'm here to help you identify the best software solution for your business.
Describe your business or idea in a few sentences.



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

You: I want to create an online store for my handmade products
Analyzing your input: 'I want to create an online store for my handmade products'
Analysis: {'raw_input': 'I want to create an online store for my handmade products', 'processed_input': 'i want to create an online store for my handmade products', 'primary_intent': 'retail', 'entities': ['want', 'product', 'handmade', 'create', 'store'], 'vagueness': 'low'}

------------------------------
RECOMMENDATION RESULTS:
------------------------------
Confidence Score: 0.6200000047683716
Recommended Platform(s): Web App, Mobile App

Key Suggested Features:
 - E-commerce Cart
 - Payment Gateway Integration (Stripe/PayPal)
 - Inventory Management System
 - Admin Dashboard
 - Promo Code Engine

Estimated Development Time: 3-5 months
Estimated Cost Range: $20,000 - $40,000

------------------------------

You: I need a delivery tracking app for my courier business
Analyzing your input: 'I need a delivery tracking app for my courier busin

In [ ]:
# Test the system with some sample inputs
test_inputs = [
    "I want to sell products online",
    "I need a delivery app",
    "I have a great idea for a social network",
    "I run a restaurant and want to take orders online"
]

engine = RecommendationEngine()

print("Testing the AI Recommendation System with sample inputs:\n")
for i, input_text in enumerate(test_inputs, 1):
    print(f"Test {i}: {input_text}")
    result = engine.get_recommendation(input_text)

    if result['status'] == 'recommendation':
        print(f"Recommended Platform: {', '.join(result['recommendation']['platform'])}")
        print(f"Confidence: {result['confidence_score']}")
    elif result['status'] == 'follow_up_required':
        print(f"Follow-up: {result['message']}")
    else:
        print(f"Result: {result['message']}")
    print("-" * 50 + "\n")

Testing the AI Recommendation System with sample inputs:

Test 1: I want to sell products online
Analyzing your input: 'I want to sell products online'
Analysis: {'raw_input': 'I want to sell products online', 'processed_input': 'i want to sell products online', 'primary_intent': 'retail', 'entities': ['want', 'sell', 'product'], 'vagueness': 'high'}
Follow-up: What specific products are you looking to sell online? Do you need features like a shopping cart or inventory management?
--------------------------------------------------

Test 2: I need a delivery app
Analyzing your input: 'I need a delivery app'
Analysis: {'raw_input': 'I need a delivery app', 'processed_input': 'i need a delivery app', 'primary_intent': 'logistics', 'entities': ['delivery', 'app', 'need'], 'vagueness': 'high'}
Follow-up: Are you looking for real-time tracking for your drivers and a dashboard for your dispatchers?
--------------------------------------------------

Test 3: I have a great idea for a social ne